HEURISTIC 1

In [25]:
import numpy as np
import random
import pandas as pd
import copy
from collections import defaultdict
from networkx import nx



In [26]:
adjacency_df=pd.DataFrame(pd.read_csv("C:/Users/Lenovo/Desktop/adjacency_df-n100-m400.csv"))

n = len(adjacency_df)

adjacency_df.columns=(range(1,(n+4)))
adjacency_df=adjacency_df.rename(columns={n+1:'Threshold',n+2:'IsInfected', n+3:'IsRisky' },errors='raise')

adjacency_df.index=(range(1,(n+1)))

m=((adjacency_df.loc[:,range(1,n+1)]>0).sum().sum())/2


In [27]:
#parameters
Blocking_Capacity = (int(n/10))*2
step_size=3

In [28]:
#sets

setV = range(1,n+1)
setS = adjacency_df.loc[(adjacency_df.loc[:, "IsInfected"] < 1),"IsInfected"].index.tolist()
setP = adjacency_df.loc[(adjacency_df.loc[:, "IsInfected"] > 0),"IsInfected"].index.tolist()
setRisky= adjacency_df.loc[(adjacency_df.loc[:, "IsRisky"] > 0),"IsRisky"].index.tolist()


In [29]:
#Preprocessing
preprocessed_matrix = adjacency_df.copy(deep=True)



In [30]:
#2 infected 
preprocessed_matrix.loc[setP,setP] = 0




In [31]:
#Sum of all edges lower than threshold (not infected)
index = preprocessed_matrix.loc[(preprocessed_matrix[setV].sum(axis=1)<((preprocessed_matrix.loc[:, "Threshold"]))) & (preprocessed_matrix.loc[:,"IsInfected"]<0.01)  , range(1,n+1)].index
preprocessed_matrix.loc[index,setV] = 0
preprocessed_matrix.loc[setV,index] = 0



In [32]:
#1 neighbor (not infected not risky)
index2 = preprocessed_matrix.loc[((preprocessed_matrix.loc[setV,setV]>0).sum(axis=1) == 1 & (preprocessed_matrix.loc[:,"IsInfected"]<0.01)) & (preprocessed_matrix.loc[:,"IsRisky"]<0.01), range(1,n+1)].index
preprocessed_matrix.loc[index2,setV] = 0
preprocessed_matrix.loc[setV,index2] = 0

In [33]:
#Percentage assignment
preprocessed_matrix.loc[:,setV]=preprocessed_matrix.loc[:,setV].div(preprocessed_matrix.loc[:,"Threshold"],axis=0)
preprocessed_matrix[preprocessed_matrix[setV] >= 1] = 1


In [34]:
#!!!!!!!!! Edges below %50 influence are deleted !!!!!!!!!!!
preprocessed_matrix[preprocessed_matrix[setV] <= 0.3] = 0

In [35]:
def convert(a):      
    rows, cols = np.where(preprocessed_matrix.loc[range(1,n+1), range(1,n+1)] > 0)
    edges = zip(rows.tolist(), cols.tolist())
    G = nx.Graph()
    G.add_edges_from(edges)
    adj_list1 = defaultdict(list)  
    for node in G.nodes():
        for each in G.neighbors(node):
            adj_list1[node+1].append(each+1)   
        
    return adj_list1

In [36]:
preprocessed_adj_list = convert(preprocessed_matrix)


In [37]:
def Find_Paths(adjList, start_node, end_node, visited, path, found_paths, path_length):
     
    # Mark the current node as visited and store in path
    visited[start_node]= True
    path.append(start_node)
    
        # If current vertex is same as destination, then print
        # current path[]
    if ((start_node == end_node)):
        new_list = path.copy()
        found_paths.append(new_list)      
    elif(len(path)<(path_length+1)): 
        # If current vertex is not destination
        # Recur for all the vertices adjacent to this vertex
         for i in adjList[start_node]:
            if visited[i] == False:
                #print(found_paths)
                Find_Paths(adjList, i, end_node, visited, path, found_paths, path_length)
                      
        # Remove current vertex from path[] and mark it as unvisited
    path.pop()
    visited[start_node]= False
   
       

In [38]:
found_paths=[]
for starting_nodes in setP:
    for ending_nodes in setRisky:
        visited=np.zeros((n+1),dtype=bool)
        path=[]
        Find_Paths(preprocessed_adj_list, starting_nodes, ending_nodes, visited, path, found_paths, step_size)  

In [39]:
found_paths

[[16, 7, 63, 10],
 [16, 8, 6, 10],
 [16, 66, 6, 10],
 [16, 8, 43, 11],
 [16, 24, 53, 11],
 [16, 24, 58, 13],
 [16, 24, 65, 13],
 [16, 24, 91, 13],
 [16, 24, 72, 15],
 [16, 7, 20, 24],
 [16, 24],
 [16, 7, 41, 25],
 [16, 8, 45, 25],
 [16, 66, 25],
 [16, 66, 41, 25],
 [16, 87, 17, 25],
 [16, 87, 4, 25],
 [16, 7, 23, 26],
 [16, 7, 26],
 [16, 8, 23, 26],
 [16, 24, 72, 26],
 [16, 66, 25, 32],
 [16, 87, 62, 32],
 [16, 7, 23, 35],
 [16, 8, 23, 35],
 [16, 87, 62, 35],
 [16, 66, 25, 36],
 [16, 66, 80, 36],
 [16, 87, 18, 36],
 [16, 87, 4, 36],
 [16, 8, 9, 37],
 [16, 24, 58, 37],
 [16, 66, 40, 37],
 [16, 66, 78, 37],
 [16, 87, 18, 37],
 [16, 87, 56, 37],
 [16, 8, 14, 40],
 [16, 66, 40],
 [16, 66, 78, 40],
 [16, 66, 80, 40],
 [16, 7, 20, 45],
 [16, 8, 14, 45],
 [16, 8, 45],
 [16, 24, 20, 45],
 [16, 24, 58, 45],
 [16, 66, 25, 45],
 [16, 7, 26, 50],
 [16, 7, 41, 50],
 [16, 8, 9, 50],
 [16, 66, 40, 50],
 [16, 66, 41, 50],
 [16, 87, 4, 50],
 [16, 24, 53],
 [16, 7, 23, 55],
 [16, 8, 23, 55],
 [16, 24, 5

In [40]:
for j in setP:
    found_paths = [[ele for ele in sub if ele != j] for sub in found_paths]

In [41]:
remaining_blocking_capacity=Blocking_Capacity
found_paths_removable=found_paths
already_blocked_nodes=[]
found_paths_np = np.concatenate([np.array(i) for i in found_paths])

In [42]:
nodes_on_paths, total_appearance = np.unique(found_paths_np, return_counts=True)

for i in setP:
    total_appearance=np.delete(total_appearance, np.where(nodes_on_paths[:] == i))
    nodes_on_paths=np.delete(nodes_on_paths, np.where(nodes_on_paths[:] == i))
#for i in setRisky:
    #total_appearance=np.delete(total_appearance, np.where(nodes_on_paths[:] == i))
    #nodes_on_paths=np.delete(nodes_on_paths, np.where(nodes_on_paths[:] == i))    


In [43]:
#nodes_on_paths[np.argpartition(total_appearance, -Blocking_Capacity)[-Blocking_Capacity:]]



In [44]:
already_blocked_nodes.append(nodes_on_paths[np.argmax(total_appearance)])
remaining_blocking_capacity -= 1

In [45]:
while (remaining_blocking_capacity>0):
    
    
    found_paths_removable = list(filter(lambda x: already_blocked_nodes[-1] not in x, found_paths_removable))
    
    if (len(found_paths_removable)<=0):
        remaining_risky=np.array(setRisky)

        for i in already_blocked_nodes:
            remaining_risky=np.delete(remaining_risky, np.where(remaining_risky[:] == i))

        if(len(remaining_risky)>0):
            if(len(remaining_risky)>remaining_blocking_capacity):
                chosen=(np.random.choice(remaining_risky,size=remaining_blocking_capacity,replace=False))  
                for items in chosen:
                    already_blocked_nodes.append(items)
            else:
                 for items in remaining_risky:
                    already_blocked_nodes.append(items)   
        
        break
    
    found_paths_removable_np = np.concatenate([np.array(i) for i in found_paths_removable])

    nodes_on_paths, total_appearance = np.unique(found_paths_removable_np, return_counts=True)

    #for i in setP:
        #total_appearance=np.delete(total_appearance, np.where(nodes_on_paths[:] == i))
        #nodes_on_paths=np.delete(nodes_on_paths, np.where(nodes_on_paths[:] == i))

    already_blocked_nodes.append(nodes_on_paths[np.argmax(total_appearance)])
    remaining_blocking_capacity -= 1


In [46]:
blocked_nodes = np.zeros(n)
#blocked_nodes[nodes_on_paths[np.argpartition(total_appearance, -Blocking_Capacity)[-Blocking_Capacity:]]-1]=1


In [47]:
blocked_nodes_df=pd.DataFrame(blocked_nodes).T
blocked_nodes_df.columns=(range(1,(n+1)))
blocked_nodes_df
blocked_nodes_df[already_blocked_nodes]=1

In [48]:
blocked_nodes_df.to_csv('Utku_blocked_nodes_df-n'+str(n)+'-m'+str(m)+'.csv',index=False)
